In [ ]:
library(tidyverse)
library(nnet)
library(purrr)
library(furrr)
library(lme4)
library(ggplot2)
library(patchwork)
library(ggbeeswarm)


options(ggrepel.max.overlaps = Inf)

In [ ]:
# xe_obj = qs::qread('230710_xenium_sct_unimapped_cca_polar_label.qs')
xe_obj_path = '_targets/objects/xe_obj_cca_td_2s'
xe_obj = qs::qread(xe_obj_path)
xe_obj@meta.data = xe_obj@meta.data %>%
mutate(predicted.polar_label = polar_label) %>%
filter(!is.na(predicted.polar_label))

xe_obj

In [ ]:
# mutate(polar_label = predicted.polar_label_2s) %>% #only for manual object
# mutate(predicted.polar_label = polar_label) %>% #only for manual object

In [ ]:
xe_obj@assays$Xenium %>% Features

In [ ]:
xe_obj %>% `[[` %>% pull(predicted.polar_label) %>% unique

In [ ]:
xe_obj %>%
`[[` %>%
rownames %>% length

In [ ]:
xe_obj@meta.data %>% colnames

In [ ]:
meta = xe_obj@meta.data

In [ ]:
summary_df = xe_obj %>% 
`[[` %>%
group_by(treatment, predicted.polar_label) %>%
summarise(n = n()) %>%
pivot_wider(names_from = treatment, values_from = n) %>%
mutate(predicted.label = predicted.polar_label %>% str_replace(fixed('.neg'), '') %>% str_replace(fixed('.none'), '') %>% str_replace(fixed('.pos'), '')) %>%
rowwise %>%
mutate(polarity = str_split(predicted.polar_label, fixed('.')) %>% unlist %>% `[[`(2)) %>%
ungroup %>%
group_by(predicted.label) %>%
mutate(FGF1_label_count = sum(FGF1)) %>%
mutate(VehPF_label_count = sum(Veh_PF)) %>%
mutate(label_diff = FGF1_label_count-VehPF_label_count) %>%
mutate(label_diff_frac = label_diff/VehPF_label_count) %>%
mutate(FGF1_minus_VehPF_polar = FGF1 - Veh_PF) %>%
relocate(FGF1_minus_VehPF_polar, .after='Veh_PF') %>%
ungroup %>%
mutate(sum_FGF1_all = sum(FGF1, na.rm = TRUE)) %>%
mutate(sum_VehPF_all = sum(Veh_PF, na.rm=TRUE)) %>%
mutate(FGF1_minus_VehPF_all = sum_FGF1_all - sum_VehPF_all) %>%
arrange(desc(abs(label_diff)))


summary_df

In [ ]:
summary_df %>% filter(str_detect(predicted.polar_label,'Oligodendrocytes'))

In [ ]:
summary_df %>% filter(str_detect(predicted.polar_label,'OPC'))

In [ ]:
summary_df %>% filter(str_detect(predicted.polar_label,'Agrp'))

In [ ]:
comparison_df = xe_obj %>% 
`[[` %>%
rownames_to_column(var = 'cell') %>%
dplyr::select(cell, sample_name, treatment, predicted.polar_label) %>%
rowwise %>%
mutate(label = predicted.polar_label %>% str_replace("\\.(pos|neg|none)$", ''),
       polarity = predicted.polar_label %>% str_split(fixed('.')) %>% unlist %>% last) %>%
mutate(treatment = factor(treatment, levels=c('Veh_PF', 'FGF1'))) %>%
mutate(polarity = factor(polarity, levels = c('none', 'neg', 'pos'))) %>% 
group_by(sample_name) %>%
mutate(sample_cell_count = n()) %>%
group_by(sample_name, label) %>%
mutate(sample_label_cell_count = n()) %>%
ungroup

comparison_df %>% print

In [ ]:
comparison_df %>% group_by(label) %>% summarise(polarities = list(polarity %>% droplevels %>% levels))

In [ ]:
df = comparison_df %>% filter(label == 'Oligodendrocytes') %>%
mutate(polarity = polarity %>% relevel(ref = 'none')) %>%
mutate(polarity =  polarity %>% droplevels)
df %>% print

In [ ]:
make_input_df = function(comparison_df_chunk){
    input_df = comparison_df_chunk %>%
        mutate(polarity = polarity %>% relevel(ref = 'none')) %>%
        mutate(polarity =  polarity %>% droplevels)
    input_df
}

In [ ]:
fit_multinom = function(input_df){
    # Fit the multinomial logistic regression model
    model <- multinom(polarity ~ treatment + sample_name, data = input_df)
#     model <- multinom(polarity ~ treatment, data = input_df)
    model
}

In [ ]:
get_multinom_coef = function(model){
    coefs = summary(model)$coefficients
    coefs
}

get_multinom_zvalues = function(model){
    zvalues = summary(model)$coefficients / summary(model)$standard.errors
    zvalues
}

get_multinom_pvalues = function(zvalues){
    pvalues = 2 * (1 - pnorm(abs(zvalues)))
    pvalues
}



In [ ]:
.get_results_long_to_wide_df = function(results, polarity_name, value.type){
    results_df = results %>% as.data.frame %>%
    set_names(polarity_name) %>%
#     set_names(input_df %>% pull(polarity) %>% levels %>% last) %>% 
    t %>% as.data.frame %>%
    mutate(value_type = value.type) %>%
    rownames_to_column %>%
    relocate(value_type, .after='polarity')
    results_df
}

In [ ]:
.get_results_already_wide_df = function(results, value.type){
    results_df = results %>% 
    as.data.frame %>%
    rownames_to_column(var = 'polarity') %>%
    mutate(value_type = value.type) %>%
    relocate(value_type, .after='polarity')
    results_df
}

In [ ]:
make_multinom_results_df = function(df){
    input_df = make_input_df(df)
    model = fit_multinom(input_df)
    zvalues = model %>% get_multinom_zvalues
    pvalues = zvalues %>% get_multinom_pvalues
    coefs = model %>% get_multinom_coef
    polarity_levels = input_df %>% pull(polarity) %>% levels
    if (length(polarity_levels) > 2){
        zvalues_df = .get_results_already_wide_df(zvalues, 'zvalue')
        pvalues_df = .get_results_already_wide_df(pvalues, 'pvalue')
        coefs_df = .get_results_already_wide_df(coefs, 'coef')
    } else {
        polarity_name = polarity_levels %>% last
        zvalues_df = .get_results_long_to_wide_df(zvalues, polarity_name, 'zvalue')
        pvalues_df = .get_results_long_to_wide_df(pvalues, polarity_name, 'pvalue')
        coefs_df = .get_results_long_to_wide_df(coefs, polarity_name, 'pvalue')
    }
    results_df = rbind(pvalues_df, zvalues_df, coefs_df) %>%
    mutate(label = input_df %>% distinct(label) %>% pull) %>%
    relocate(label)
    results_df
}

In [ ]:
make_multinom_results_df_safe = function(df){
    results_df = tryCatch(
        make_multinom_results_df(df),
        error = function(e) data.frame()
    )
    results_df
}

In [ ]:
#no idea why I removed this
# comparison_df_f = comparison_df %>%
# filter(!(sample_name == 'D2' & label == 'Agrp'))

# entire comparison_df
comparison_df_f = comparison_df 

In [ ]:
#no idea why I removed this
# comparison_df_f = comparison_df %>%
# filter(!(sample_name == 'C2' & label == 'Rgs16_Dlx1'))
comparison_df_f = comparison_df 
# entire comparison_df


In [ ]:
# Group the dataframe by label and fit glmer model in parallel
results_df_list <- comparison_df_f %>%
  group_split(label) %>%
  map(function(subset) {
      result_list = subset %>% make_multinom_results_df_safe
      result_list
  })

results_df = results_df_list %>% bind_rows


In [ ]:
results_df %>% head # all Agrp

In [ ]:
polarity_sig_df = results_df %>% 
select(label, polarity, treatmentFGF1, value_type) %>% distinct %>%
# group_by(label, polarity) %>%
pivot_wider(names_from = 'value_type', values_from = 'treatmentFGF1') %>%
rename(pval = pvalue, z = zvalue) %>%
ungroup %>%
mutate(pval_corrected = p.adjust(pval, method = "BH")) %>%
arrange(label) %>% 
mutate(significant = case_when(pval < 0.05 ~ '**',
                                TRUE ~ '')) %>%
mutate(significant_adj = case_when(pval_corrected < 0.05 ~ '**',
                               TRUE ~ '')) %>%
relocate(pval, coef, z, significant, significant_adj, .after='polarity') %>%
mutate(z = case_when(abs(z) > 8 ~ sign(z)*8,
                     TRUE ~ z))

polarity_sig_df

In [ ]:
get_glmer_pvalue = function(model){
    model_summary = summary(model)
    pvalue = model_summary %>% 
        `[[`('coefficients') %>%
        as.data.frame %>%
        dplyr::select(any_of('Pr(>|z|)')) %>%
        rownames_to_column %>%
        filter(rowname == 'treatmentFGF1') %>%
        pull('Pr(>|z|)')
    pvalue
}

get_glmer_coef = function(model){
    model_summary = summary(model)
    coef = model_summary %>% 
        `[[`('coefficients') %>%
        as.data.frame %>%
        dplyr::select(any_of('Estimate')) %>%
        rownames_to_column %>%
        filter(rowname == 'treatmentFGF1') %>%
        pull('Estimate')
    coef
}

In [ ]:
make_result = function(subset){
    result_name = subset %>% distinct(label) %>% pull
    model = tryCatch(
      glmer(polarity ~ treatment + (1 | sample_name), data = subset, family = binomial),
      error = function(e) NA
    )
    pval = tryCatch(
               get_glmer_pvalue(model),
               error = function(e) 1
           )
    coef = tryCatch(
           get_glmer_coef(model),
           error = function(e) NA
       )
    result_list = list(name = result_name,
                       pval = pval,
                       coef = coef,
                       model = model)
    result_list
}

In [ ]:


# Group the dataframe by label and fit glmer model in parallel
models <- comparison_df %>%
  group_split(label) %>%
  map(function(subset) {
      result_list = subset %>% make_result
      result_list
  })


In [ ]:


# Combine models into a tibble
result <- map_dfr(models, ~ tibble(label = .x$name, pval = .x$pval, coef = .x$coef))

# Print the result
result %>% arrange(pval)


In [ ]:
make_glmer_result_ncells = function(comparison_df, current_ct){
    comparison_df_1vr = comparison_df %>%
        mutate(test_label = case_when(label == current_ct ~ current_ct,
                                      TRUE ~ 'other')) %>%
        mutate(test_label = factor(test_label, levels=c('other', current_ct)))
    print(dim(comparison_df_1vr))
    model = tryCatch(
      glmer(test_label ~ treatment + (1 | sample_name), data = comparison_df_1vr, family = binomial),
      error = function(e) NA
    )
    pval = tryCatch(
               get_glmer_pvalue(model),
               error = function(e) 1
           )
    coef = tryCatch(
           get_glmer_coef(model),
           error = function(e) NA
       )
    result_list = list(name = current_ct,
                       pval = pval,
                       coef = coef,
                       model = model)
    result_list
}

In [ ]:
make_glmer_result_ncells = function(comparison_df, current_ct){
    comparison_df_1vr = comparison_df %>%
        mutate(test_label = case_when(label == current_ct ~ current_ct,
                                      TRUE ~ 'other')) %>%
        mutate(test_label = factor(test_label, levels=c('other', current_ct)))
    print(dim(comparison_df_1vr))
    model = tryCatch(
      glmer(test_label ~ treatment + (1 | sample_name), data = comparison_df_1vr, family = binomial),
      error = function(e) NA
    )
    pval = tryCatch(
               get_glmer_pvalue(model),
               error = function(e) 1
           )
    coef = tryCatch(
           get_glmer_coef(model),
           error = function(e) NA
       )
    zscores = tryCatch(
           summary(model)$coefficients[,"z value"][[2]],
           error = function(e) NA
       )
    result_list = list(name = current_ct,
                       pval = pval,
                       coef = coef,
                       zscores = zscores,
                       model = model)
    result_list
}


In [ ]:
comparison_df %>% head

In [ ]:
labels <- unique(comparison_df$label)

# Group the dataframe by label and fit glmer model in parallel
models = map(labels, function(current_ct) {
  results_list = make_glmer_result_ncells(comparison_df, current_ct)
  results_list
})

In [ ]:
comparison_df

In [ ]:
current_ct = 'OPC'

comparison_df_1vr = comparison_df %>%
        mutate(test_label = case_when(label == current_ct ~ current_ct,
                                      TRUE ~ 'other')) %>%
        mutate(test_label = factor(test_label, levels=c('other', current_ct)))
comparison_df_1vr %>% head

In [ ]:
glmer(test_label ~ treatment + (1 | sample_name), data = comparison_df_1vr, family = binomial)

In [ ]:
sumstats_comparison = comparison_df %>% 
group_by(sample_name, treatment, label) %>% 
summarise(n=n()) %>% 
group_by(label, treatment) %>% 
summarise(median = median(n), mean=mean(n), min=min(n), max=max(n)) 

# sumstats_comparison %>% head

In [ ]:
sumstats_comparison %>% group_by(label) %>% summarise(max_group_median = max(median))

In [ ]:
# Combine models into a tibble
ncells_result <- map_dfr(models, ~ tibble(label = .x$name, pval = .x$pval, coef = .x$coef, zscore = .x$zscores))


# Print the result
ncells_sig_df = ncells_result %>% 
arrange(pval) %>% 
mutate(pval_corrected = p.adjust(pval, method = "BH")) %>%
mutate(significant = case_when(pval < 0.05 ~ '**',
                               TRUE ~ '')) %>%
mutate(significant_adj = case_when(pval_corrected < 0.05 ~ '**',
                               TRUE ~ ''))

ncells_sig_df %>% head(15)

In [ ]:
agg_data = comparison_df %>%
group_by(sample_name, treatment, label) %>%
summarise(cell_count = n())

agg_data %>% print

In [ ]:
wilcox_test_result <- wilcox.test(cell_count ~ treatment, data = agg_data)
wilcox_test_result

In [ ]:
wilcox_test_result$p.value

In [ ]:
wilcox_test_result$statistic

In [ ]:
make_result_wilcox = function(subset){
    result_name = subset %>% distinct(label) %>% pull
    pval = tryCatch(
      wilcox.test(cell_count ~ treatment, data = subset)$p.value, 
      error = function(e) NA
    )
    result_list = list(name = result_name,
                       pval = pval)
    result_list
}

In [ ]:
# Set up parallel processing
# plan(multisession, workers=40)  # Adjust according to your system capabilities

# Group the dataframe by label and fit glmer model in parallel
models_wilcox <- agg_data %>% group_by(label) %>% group_split %>%
  map(function(subset) {
      result_list = subset %>% make_result_wilcox
      result_list
  })


In [ ]:
wilcox_tibble = tibble(
  name = map_chr(models_wilcox, "name"),
  pval = map_dbl(models_wilcox, "pval")
) %>%
arrange(pval)

wilcox_tibble

In [ ]:
xe_obj@images %>% names

In [ ]:
xe_obj %>% `[[` %>% colnames

In [ ]:
xe_obj %>% `[[` %>% head

In [ ]:
xe_obj@images %>% names %>% dput

In [ ]:
xe_obj_merged = qs::qread('_targets/objects/obj_merged')
xe_obj_merged

In [ ]:
fov_names = xe_obj_merged@images %>% names
centroids_df <- purrr::map_df(fov_names, function(x) GetTissueCoordinates(xe_obj_merged@images[[x]], which='centroids')) %>%
rename(centroid_x = x, centroid_y = y)

centroids_df %>% dim
centroids_df %>% head

In [ ]:
coord_comparison_df = comparison_df %>%
merge(centroids_df, by='cell')

coord_comparison_df %>% head

In [ ]:
coord_comparison_df %>% dim

In [ ]:
options(repr.plot.width=6, repr.plot.height=14, repr.plot.res=200)
plot_single_slice_label = function(coord_comparison_df, sample, cell_type, pt_size=0.1){
    ggp = coord_comparison_df %>%
        filter(sample_name == sample) %>%
        mutate(this_cell_type = case_when(label == cell_type ~ !!cell_type,
                                          TRUE ~ 'other'),
               polarity_factor = factor(polarity, levels = c('none', 'neg', 'pos')),
               cell_type_factor = factor(label == cell_type, levels = c(FALSE, TRUE)),
               cell_color = ifelse(this_cell_type == 'other', 'other', as.character(polarity))) %>%
        arrange(cell_type_factor, polarity_factor) %>%
        ggplot(aes(x = centroid_y, y = centroid_x, color = cell_color, alpha = this_cell_type, shape = this_cell_type)) + 
          geom_point(size=pt_size) +
          xlim(0, 17000) +
          ylim(900, 10900) +
          scale_color_manual(values = c('none' =  "#DAA520",
                                        'pos' = "#2166ac",
                                        'neg' = "#b2182b",
                                        'other' = "#a1a1a1")) +  # green for "other"
          scale_shape_manual(values = setNames(c("other" = 16, "this" = 16), c("other", cell_type))) +
          scale_alpha_manual(values = setNames(c("other" = 0.1, "this" = 1), c("other", cell_type))) +
          coord_fixed() + 
          theme_light() +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank()    # Remove y-axis labels
        )
    
    ggp
}


In [ ]:
comparison_df %>% filter(sample_name == 'A2') %>% distinct(label) %>% pull

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_single_slice_label(coord_comparison_df, 'A2', 'Agrp')

In [ ]:
xe_obj %>% `[[` %>% distinct(labels) %>% filter(str_detect(labels, 'Pomc'))

In [ ]:
plot_all_samples_polarity = function(coord_comparison_df, cell_type){
    p1 = plot_single_slice_label(coord_comparison_df, 'A1', cell_type)
    p2 = plot_single_slice_label(coord_comparison_df, 'A2', cell_type)
    p3 = plot_single_slice_label(coord_comparison_df, 'B1', cell_type)
    p4 = plot_single_slice_label(coord_comparison_df, 'B2', cell_type)
    p5 = plot_single_slice_label(coord_comparison_df, 'C1', cell_type)
    p6 = plot_single_slice_label(coord_comparison_df, 'C2', cell_type)
    p7 = plot_single_slice_label(coord_comparison_df, 'D1', cell_type)
    p8 = plot_single_slice_label(coord_comparison_df, 'D2', cell_type)
    
    layout <- '
    AE
    BF
    CG
    DH
    '

    ggp = wrap_plots(A = p1,
               B = p2,
               C = p3,
               D = p4,
               E = p5,
               F = p6,
               G = p7,
               H = p8,
        design = layout) +
    plot_layout(guides = 'collect') 

    ggp
}

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_all_samples_polarity(coord_comparison_df, 'Agrp')

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_all_samples_polarity(coord_comparison_df, 'OPC')

In [ ]:
plot_all_samples_polarity(coord_comparison_df, 'Oligodendrocytes')

In [ ]:
polarity_sig_df_wide = polarity_sig_df %>%
  pivot_wider(names_from = polarity, 
              values_from = c(pval, coef, z, significant, significant_adj, pval_corrected),
              names_sep = ".")

polarity_sig_df_wide

In [ ]:
sig_df = ncells_sig_df %>%
left_join(polarity_sig_df_wide, by='label')

# mutate(z.neg = ifelse(is.na(z.neg), 0, z.neg),
#        z.pos = ifelse(is.na(z.pos), 0, z.pos),
#        pval_corrected.neg = ifelse(is.na(pval_corrected.neg), 1, pval_corrected.neg),
#        pval_corrected.pos = ifelse(is.na(pval_corrected.pos), 1, pval_corrected.pos)) %>%
# mutate(sort_col = pval_corrected * pval_corrected.neg * pval_corrected.pos) %>%
# mutate(sort_col = sign(zscore) * abs(zscore * z.pos * z.neg)) %>%

counts_for_sig = summary_df %>% filter(predicted.label %in% c(sig_df %>% filter(pval_corrected < 0.05 | pval_corrected.neg < 0.05 | pval_corrected.pos < 0.05) %>% pull (label))                            
                     ) %>%
select(predicted.label, polarity, FGF1_minus_VehPF_polar, label_diff) %>%
  # Use pivot_wider to spread the data
  pivot_wider(
    names_from = polarity, 
    values_from = c(FGF1_minus_VehPF_polar, label_diff),
    names_sep = "_"
  ) %>%
  select(-label_diff_neg, -label_diff_pos) %>%
  rename(label = predicted.label)

sig_df = sig_df %>%
left_join(counts_for_sig, by = 'label') %>%
mutate(sort_col = zscore) %>%
arrange(desc(sort_col)) %>%
mutate(label = factor(label, levels=label))

sig_df %>% head

In [ ]:
summary_df %>% filter(predicted.label %in% c(sig_df %>% filter(pval_corrected < 0.05 | pval_corrected.neg < 0.05 | pval_corrected.pos < 0.05) %>% pull (label)))

In [ ]:
sig_df %>% filter(label == 'Ependymal_cells')

In [ ]:
options(repr.plot.width=3.5, repr.plot.height=11)

# Convert 'label' column to a factor
sig_df$label <- as.factor(sig_df$label)

# Define conditions for colors
sig_df$zscore_col <- ifelse(sig_df$zscore > 0 & sig_df$significant_adj == '**' & abs(sig_df$label_diff_none) >= 20, "#2166ac",
                            ifelse(sig_df$zscore < 0 & sig_df$significant_adj == '**' & abs(sig_df$label_diff_none) >= 20, "#b2182b", "#a1a1a1"))
sig_df$zpos_col <- ifelse(sig_df$z.pos > 0 & sig_df$significant_adj.pos == '**' & abs(sig_df$FGF1_minus_VehPF_polar_pos) >= 20, "#2166ac",
                         ifelse(sig_df$z.pos < 0 & sig_df$significant_adj.pos == '**' & abs(sig_df$FGF1_minus_VehPF_polar_pos) >= 20, "#b2182b", "#a1a1a1"))
sig_df$zneg_col <- ifelse(sig_df$z.neg > 0 & sig_df$significant_adj.neg == '**' & abs(sig_df$FGF1_minus_VehPF_polar_neg) >= 20, "#2166ac",
                         ifelse(sig_df$z.neg < 0 & sig_df$significant_adj.neg == '**' & abs(sig_df$FGF1_minus_VehPF_polar_neg) >= 20, "#b2182b", "#a1a1a1"))

# Create the scatter plot
ggp_sig = ggplot(sig_df) +
  # Add a scatter plot for 'zscore'
  geom_point(aes(y = label, x = zscore, color = zscore_col), shape = 'o', size = 5) +
  # Add a scatter plot for 'z.pos'
  geom_point(aes(y = label, x = z.pos, color = zpos_col), shape = '+', size = 5) +
  # Add a scatter plot for 'z.neg'
  geom_point(aes(y = label, x = z.neg, color = zneg_col), shape = '|', size = 5) +
  # Specify the colors for each condition
  scale_color_identity() +
  # Set the x-axis limits
  scale_x_continuous(limits = c(-8, 8)) +
  # Add the plot title and labels
  labs(x = "Z") +
  # Set the plot theme
  theme_minimal()

ggp_sig

In [ ]:
summary_df %>% filter(predicted.label %in% c(sig_df %>% filter(pval_corrected < 0.05 | pval_corrected.neg < 0.05 | pval_corrected.pos < 0.05) %>% pull (label))
                                            
                     ) %>%
select(predicted.label, polarity, FGF1_minus_VehPF_polar, label_diff) %>%
rename(label = predicted.label) %>%
print

In [ ]:
summary_df %>% filter(predicted.label %in% c(sig_df %>% filter(pval_corrected < 0.05 | pval_corrected.neg < 0.05 | pval_corrected.pos < 0.05) %>% pull (label))
                                            
                     )

In [ ]:
sig_df %>% filter(pval_corrected < 0.05 | pval_corrected.neg < 0.05 | pval_corrected.pos < 0.05) 

In [ ]:
sig_df %>% filter(label == 'Agrp') %>% select(label, zscore,  zscore_col, z.neg, zneg_col, z.pos,  zpos_col)

In [ ]:
plot_all_samples_polarity_2row = function(coord_comparison_df, cell_type, pt_size=0.2){
    p1 = plot_single_slice_label(coord_comparison_df, 'A1', cell_type, pt_size=pt_size) + theme(legend.position = "none") + labs(y = paste0(cell_type, '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p2 = plot_single_slice_label(coord_comparison_df, 'A2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p3 = plot_single_slice_label(coord_comparison_df, 'B1', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p4 = plot_single_slice_label(coord_comparison_df, 'B2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p5 = plot_single_slice_label(coord_comparison_df, 'C1', cell_type, pt_size=pt_size) + theme(legend.position = "none") + labs(y = paste0(cell_type, '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p6 = plot_single_slice_label(coord_comparison_df, 'C2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p7 = plot_single_slice_label(coord_comparison_df, 'D1', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p8 = plot_single_slice_label(coord_comparison_df, 'D2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    label_plot <- ggplot() +
      labs(y = cell_type) +
      theme_void()

    
    layout <- '
    ABCD
    EFGH
    '

    ggp = wrap_plots(A = p1,
               B = p2,
               C = p3,
               D = p4,
               E = p5,
               F = p6,
               G = p7,
               H = p8,
        design = layout) +
    plot_layout(guides = 'collect') & 
      theme(plot.margin = margin(0, 0, 0, 0))

    ggp
}

In [ ]:
p_Microglia = plot_all_samples_polarity_2row(coord_comparison_df, 'Microglia')
p_MOL = plot_all_samples_polarity_2row(coord_comparison_df, 'Oligodendrocytes')
p_Trh_Lef1 = plot_all_samples_polarity_2row(coord_comparison_df, 'Trh_Lef1')
p_Astrocytes = plot_all_samples_polarity_2row(coord_comparison_df, 'Astrocytes')
p_a1_Tany__Astrocytes = plot_all_samples_polarity_2row(coord_comparison_df, 'a1_Tany__Astrocytes')
p_a2_Tanycytes = plot_all_samples_polarity_2row(coord_comparison_df, 'a2_Tanycytes')
p_NFOL = plot_all_samples_polarity_2row(coord_comparison_df, 'NFOL')
p_Agrp = plot_all_samples_polarity_2row(coord_comparison_df, 'Agrp')
p_Rgs16_Dlx1 = plot_all_samples_polarity_2row(coord_comparison_df, 'Rgs16_Dlx1')

In [ ]:
options(repr.plot.width=8.5, repr.plot.height=11)
layout <- '
ABBBBBB
ACCCCCC
ADDDDDD
AEEEEEE
AFFFFFF
AGGGGGG
AHHHHHH
'

ggp = wrap_plots(A = ggp_sig,
           B = p_Microglia,
           C = p_MOL,
           D = p_Trh_Lef1,
           E = p_Astrocytes,
           F = p_a1_Tany__Astrocytes,
           G = p_a2_Tanycytes,
           H = p_NFOL,
    design = layout) +
plot_layout(guides = 'auto') 

In [ ]:
options(repr.plot.width=8.5, repr.plot.height=11)
layout <- '
ABBBBBB
ACCCCCC
ADDDDDD
AEEEEEE
AGGGGGG
'

ggp = wrap_plots(A = ggp_sig,
           B = p_Rgs16_Dlx1 ,
           C = p_Agrp,
           D = p_MOL,
           E = p_a1_Tany__Astrocytes,
           G = p_NFOL,
    design = layout) +
plot_layout(guides = 'auto') 

In [ ]:
ggp %>% ggsave("spatial_sig_plots.pdf", ., width = 7, height = 9)

In [ ]:
ggp %>% ggsave("spatial_sig_plots.png", ., width = 8.5, height = 11)

In [ ]:
plot_single_slice_label_ct = function(coord_comparison_df, sample, cell_type, pt_size=0.1){
    ggp = coord_comparison_df %>%
        filter(sample_name == sample) %>%
        mutate(this_cell_type = case_when(label == cell_type ~ !!cell_type,
                                          TRUE ~ 'other'),
               polarity_factor = factor(polarity, levels = c('none', 'neg', 'pos')),
               cell_type_factor = factor(label == cell_type, levels = c(FALSE, TRUE)),
               cell_color = ifelse(this_cell_type == 'other', 'other', as.character(polarity))) %>%
        arrange(cell_type_factor, polarity_factor) %>%
        ggplot(aes(x = centroid_y, y = centroid_x, color = cell_color, alpha = this_cell_type, shape = this_cell_type)) + 
          geom_point(size=pt_size) +
          xlim(0, 17000) +
          ylim(900, 10900) +
          scale_color_manual(values = c('none' =  "#000000",
                                        'pos' = "#000000",
                                        'neg' = "#000000",
                                        'other' = "#a1a1a1")) +  # green for "other"
          scale_shape_manual(values = setNames(c("other" = 16, "this" = 16), c("other", cell_type))) +
          scale_alpha_manual(values = setNames(c("other" = 0.1, "this" = 1), c("other", cell_type))) +
          coord_fixed() + 
          theme_light() +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank()    # Remove y-axis labels
        )
    
    ggp
}


In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_single_slice_label_ct(coord_comparison_df, 'A1', 'Agrp')

In [ ]:
plot_all_samples_ct_2col = function(coord_comparison_df, cell_type, pt_size=0.2){
    p1 = plot_single_slice_label_ct(coord_comparison_df, 'A1', cell_type, pt_size=pt_size) + theme(legend.position = "none") #+ labs(y = paste0(cell_type, '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p2 = plot_single_slice_label_ct(coord_comparison_df, 'A2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p3 = plot_single_slice_label_ct(coord_comparison_df, 'B1', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p4 = plot_single_slice_label_ct(coord_comparison_df, 'B2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p5 = plot_single_slice_label_ct(coord_comparison_df, 'C1', cell_type, pt_size=pt_size) + theme(legend.position = "none") #+ labs(y = paste0(cell_type, '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p6 = plot_single_slice_label_ct(coord_comparison_df, 'C2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p7 = plot_single_slice_label_ct(coord_comparison_df, 'D1', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    p8 = plot_single_slice_label_ct(coord_comparison_df, 'D2', cell_type, pt_size=pt_size) + theme(legend.position = "none")
    label_plot <- ggplot() +
      labs(y = cell_type) +
      theme_void()

    
    layout <- '
AE
BF
CG
DH
'

    ggp = wrap_plots(A = p1,
               B = p2,
               C = p3,
               D = p4,
               E = p5,
               F = p6,
               G = p7,
               H = p8,
        design = layout) +
    plot_layout(guides = 'collect') & 
      theme(plot.margin = margin(0, 0, 0, 0))

    ggp
}

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_all_samples_ct_2col(coord_comparison_df, 'Agrp', pt_size=0.3)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_all_samples_ct_2col(coord_comparison_df, 'Tmem215__Dach2', pt_size=0.3)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6, repr.plot.res=200)
plot_all_samples_ct_2col(coord_comparison_df, 'Astrocytes', pt_size=0.3)

In [ ]:
plot_all_samples_ct_2col(coord_comparison_df, 'Htr3b', pt_size=0.6)

In [ ]:
plot_all_samples_ct_2col(coord_comparison_df, 'a1_Tanycytes')

In [ ]:
plot_all_samples_ct_2col(coord_comparison_df, 'a2_Tanycytes')

In [ ]:
plot_all_samples_ct_2col(coord_comparison_df, 'b1_Tanycytes')

In [ ]:
plot_all_samples_ct_2col(coord_comparison_df, 'b2_Tanycytes')

In [ ]:
plot_single_slice_label_ct_tany = function(coord_comparison_df, sample, pt_size=0.1){
    tany_regex = "^(a1_Tanycytes|a2_Tanycytes|b1_Tanycytes|b2_Tanycytes)$"
    ggp = coord_comparison_df %>%
        filter(sample_name == sample) %>%
        mutate(this_cell_type = case_when(str_detect(label, tany_regex) ~ label,
                                          TRUE ~ 'other'),
               cell_type_factor = factor(str_detect(label, tany_regex), levels = c(FALSE, TRUE)),
               cell_color = ifelse(this_cell_type == 'other', 'other', as.character(this_cell_type))) %>%
        arrange(cell_type_factor) %>%
        ggplot(aes(x = centroid_y, y = centroid_x, color = cell_color, alpha = this_cell_type, shape = this_cell_type)) + 
          geom_point(size=pt_size) +
          xlim(0, 17000) +
          ylim(900, 10900) +
          scale_color_manual(name = 'cell type',
                             values = c('a2_Tanycytes' =  "#e7d4e8",
                                        'a1_Tanycytes' = "#af8dc3",
                                        'b1_Tanycytes' = "#7fbf7b",
                                        'b2_Tanycytes' = "#1b7837",
                                        'other' = "#a1a1a1")) +  # green for "other"
          scale_shape_manual(values = c("other" = 16,
                                        'a2_Tanycytes' = 16,
                                        'a1_Tanycytes' = 16,
                                        'b1_Tanycytes' = 16,
                                        'b2_Tanycytes' = 16),
                             guide = 'none') +
          scale_alpha_manual(values = c("other" = 0.1,
                                        'a2_Tanycytes' = 1,
                                        'a1_Tanycytes' = 1,
                                        'b1_Tanycytes' = 1,
                                        'b2_Tanycytes' = 1),
                             guide='none') +
          coord_fixed() + 
          theme_light() +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank(),    # Remove y-axis labels
              legend.position = "right", 
              legend.key.size = unit(0.3, "cm"),
              legend.text = element_text(lineheight=0.5, size = 6)

        )
    
    ggp
}


In [ ]:
plot_single_slice_label_ct_tany(coord_comparison_df, 'A1')

In [ ]:
plot_all_samples_ct_tany = function(coord_comparison_df, pt_size=0.1, plot_shape='2col'){
    p1 = plot_single_slice_label_ct_tany(coord_comparison_df, 'A1', pt_size=pt_size) #+ theme(legend.position = "bottom") #+ labs(y = paste0(cell_type, '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p2 = plot_single_slice_label_ct_tany(coord_comparison_df, 'A2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p3 = plot_single_slice_label_ct_tany(coord_comparison_df, 'B1', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p4 = plot_single_slice_label_ct_tany(coord_comparison_df, 'B2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p5 = plot_single_slice_label_ct_tany(coord_comparison_df, 'C1', pt_size=pt_size) #+ theme(legend.position = "bottom") #+ labs(y = paste0(cell_type, '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p6 = plot_single_slice_label_ct_tany(coord_comparison_df, 'C2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p7 = plot_single_slice_label_ct_tany(coord_comparison_df, 'D1', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p8 = plot_single_slice_label_ct_tany(coord_comparison_df, 'D2', pt_size=pt_size) #+ theme(legend.position = "bottom")

if (plot_shape == '2col'){
        layout <- '
AE
BF
CG
DH
'
}
if (plot_shape == '2row'){
            layout <- '
ABCD
EFGH
'
p1 = p1 + labs(y = paste0('Tanycytes', '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
p5 = p5 + labs(y = paste0('Tanycytes', '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
}


    ggp = wrap_plots(A = p1,
               B = p2,
               C = p3,
               D = p4,
               E = p5,
               F = p6,
               G = p7,
               H = p8,
        design = layout) +
    plot_layout(guides = 'collect') & 
      theme(plot.margin = margin(0, 0, 0, 0),
#            legend.position = "bottom")
                       legend.position = "right")

    ggp
}

In [ ]:
options(repr.plot.width=7, repr.plot.height=4)
tany_plot = plot_all_samples_ct_tany(coord_comparison_df, pt_size=0.2, plot_shape='2row')
tany_plot

In [ ]:
plot_single_slice_label_ct_neurons = function(coord_comparison_df, sample, pt_size=0.1){
    tany_regex = "^(Agrp|Sst_Unc13c__Agrp|Pomc_Lepr|Pomc_Glipr1__Slc17a6_Trhr|Htr3b|Nr5a1_Bdnf)$"
    ggp = coord_comparison_df %>%
        filter(sample_name == sample) %>%
        mutate(this_cell_type = case_when(str_detect(label, tany_regex) ~ label,
                                          TRUE ~ 'other'),
               cell_type_factor = factor(str_detect(label, tany_regex), levels = c(FALSE, TRUE)),
               cell_color = ifelse(this_cell_type == 'other', 'other', as.character(this_cell_type))) %>%
        mutate(cell_color = factor(cell_color, levels=c("other", "Pomc_Glipr1__Slc17a6_Trhr", "Sst_Unc13c__Agrp",  "Htr3b", "Pomc_Lepr", "Agrp", "Nr5a1_Bdnf"))) %>%
        arrange(cell_color) %>%
        ggplot(aes(x = centroid_y, y = centroid_x, color = cell_color, alpha = this_cell_type, shape = this_cell_type)) + 
          geom_point(size=pt_size) +
          xlim(0, 17000) +
          ylim(900, 10900) +
          scale_color_manual(name = 'cell type',
                             values = c('Sst_Unc13c__Agrp' =  "#a6cee3",
                                        'Agrp' = "#1f78b4",
#                                         'Pomc_Glipr1__Slc17a6_Trhr' = "#33a02c", #"#b2df8a",
                                        'Pomc_Lepr' = "#33a02c",
                                        'Htr3b' = '#e31a1c',
                                        'Nr5a1_Bdnf' = '#fdbf6f',
                                        'other' = "#a1a1a1")) +  # green for "other"
          scale_shape_manual(values = c('Agrp' =  16,
                                        'Sst_Unc13c__Agrp' = 16,
                                        'Pomc_Lepr' = 16,
#                                         'Pomc_Glipr1__Slc17a6_Trhr' = 16,
                                        'Htr3b' = 16,
                                        'Nr5a1_Bdnf' = 16,
                                        'other' = 16), guide = 'none') +
          scale_alpha_manual(values = c('Agrp' =  1,
                                        'Sst_Unc13c__Agrp' = 1,
                                        'Pomc_Lepr' = 1,
#                                         'Pomc_Glipr1__Slc17a6_Trhr' = 1,
                                        'Htr3b' = 1,
                                        'Nr5a1_Bdnf' = 1,
                                        'other' = 0.1), guide = 'none') +
          coord_fixed() + 
          theme_light() +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank(),    # Remove y-axis labels
              legend.position = "right", 
              legend.key.size = unit(0.3, "cm"),
              legend.text = element_text(lineheight=0.5, size = 6)
        )
    
    ggp
}


In [ ]:
names(c('Sst_Unc13c__Agrp' =  "#a6cee3",
                                        'Agrp' = "#1f78b4",
                                        'Pomc_Glipr1__Slc17a6_Trhr' = "#33a02c", #"#b2df8a",
#                                         'Pomc_Lepr' = "#33a02c",
                                        'Htr3b' = '#e31a1c',
                                        'Nr5a1_Bdnf' = '#fdbf6f',
                                        'other' = "#a1a1a1")) %>% rev %>%dput

In [ ]:
plot_all_samples_ct_neurons = function(coord_comparison_df, pt_size=0.2, plot_shape='2col'){
    p1 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'A1', pt_size=pt_size) #+ theme(legend.position = "bottom") #+ labs(y = paste0(cell_type, '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p2 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'A2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p3 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'B1', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p4 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'B2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p5 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'C1', pt_size=pt_size) #+ theme(legend.position = "bottom") #+ labs(y = paste0(cell_type, '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
    p6 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'C2', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p7 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'D1', pt_size=pt_size) #+ theme(legend.position = "bottom")
    p8 = plot_single_slice_label_ct_neurons(coord_comparison_df, 'D2', pt_size=pt_size) #+ theme(legend.position = "bottom")

if (plot_shape == '2col'){
        layout <- '
AE
BF
CG
DH
'
}
if (plot_shape == '2row'){
            layout <- '
ABCD
EFGH
'
p1 = p1 + labs(y = paste0('Neurons', '\nVehPF')) + theme(axis.title.y = element_text(size = 8, angle = 90))
p5 = p5 + labs(y = paste0('Neurons', '\nFGF1')) + theme(axis.title.y = element_text(size = 8, angle = 90))
}


    ggp = wrap_plots(A = p1,
               B = p2,
               C = p3,
               D = p4,
               E = p5,
               F = p6,
               G = p7,
               H = p8,
        design = layout) +
    plot_layout(guides = 'collect') & 
      theme(plot.margin = margin(0, 0, 0, 0),
#            legend.position = "bottom")
            )
    ggp
}

In [ ]:
options(repr.plot.width=7, repr.plot.height=4)
neurons_plot = plot_all_samples_ct_neurons(coord_comparison_df, pt_size=0.2, plot_shape='2row')
neurons_plot

In [ ]:
obj_fgf1 = qs::qread('_targets/objects/obj_d5_01')
obj_fgf1

In [ ]:
xe_obj = qs::qread(xe_obj_path)

xe_obj@meta.data = xe_obj@meta.data %>%
mutate(predicted.polar_label = polar_label)

xe_obj@meta.data = xe_obj %>% `[[` %>%
mutate(base_label = predicted.polar_label %>% str_replace(fixed('.neg'), '') %>% str_replace(fixed('.none'), '') %>% str_replace(fixed('.pos'), ''))

xe_obj@meta.data = xe_obj@meta.data %>% mutate(base_label = str_replace(base_label, fixed('__'), '\n'))

In [ ]:
xe_obj %>% `[[` %>% distinct(base_label) %>% pull(base_label)

In [ ]:
obj_fgf1 %>% `[[` %>% colnames

In [ ]:
library(Seurat)

xe_umap = DimPlot(xe_obj, reduction = "umap", group.by = "base_label", label = TRUE, repel = TRUE, label.size=2, raster = FALSE) + theme(legend.position = "none", plot.title=element_text(size=10)) + labs(title = "Spatial on snRNAseq reference UMAP") +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank())    # Remove y-axis labels
xe_ref.umap = DimPlot(xe_obj, reduction = "ref.umap", group.by = "base_label", label = TRUE, repel = TRUE, label.size=2, raster = FALSE) + theme(legend.position = "none") + labs(title = "Spatial on snRNAseq reference UMAP") +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank())
sc_ref.umap = DimPlot(obj_fgf1, reduction = "umap", group.by = "labels", label = TRUE, repel = TRUE, label.size=2, raster = FALSE) + theme(legend.position = "none") + labs(title = "snRNAseq reference") +
          theme(axis.title.x = element_blank(),
                axis.title.y = element_blank()) + 
        theme(axis.text.x = element_blank(),   # Remove x-axis labels
              axis.text.y = element_blank())
options(repr.plot.width=30, repr.plot.height=10)
umap_plots = xe_umap + xe_ref.umap #+ sc_ref.umap
umap_plots = umap_plots + theme(legend.position = "none")

In [ ]:
xe_umap

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=3.5)
layout <- '
B
B
C
C
'
ggp = wrap_plots(B = neurons_plot,
                 C = tany_plot,
        design = layout) +
    plot_layout(guides = 'collect') & 
      theme(plot.margin = margin(0, 0, 0, 0))

    ggp

In [ ]:
ggp %>% ggsave("label_transfer_spatial_only.pdf", ., width = 7.5, height = 3.5)

In [ ]:
markers = qs::qread('_targets/objects/polar_label_markers_2s')
markers

In [ ]:
markers %>% filter(cluster == 'Agrp.pos')

In [ ]:
markers %>% filter(cluster == 'Agrp.neg')

In [ ]:
obj_fgf1_neurons = qs::qread('_targets/objects/obj_fgf1')
obj_fgf1_neurons

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=7.5)
Idents(obj_fgf1_neurons) = obj_fgf1_neurons@meta.data$polar_label
agrp_cells = obj_fgf1_neurons %>% `[[` %>% filter(labels == 'Agrp') %>% rownames
FeaturePlot(object = obj_fgf1_neurons, features = 'Pcdh9', cells = agrp_cells, label = TRUE, order=TRUE)

In [ ]:
options(repr.plot.width=15, repr.plot.height=3)
obj_fgf1_neurons %>%
subset(cells = agrp_cells) %>%
VlnPlot(features= markers %>% filter(cluster == 'Agrp.neg') %>% head(10) %>% pull(gene), ncol=10)

In [ ]:
options(repr.plot.width=15, repr.plot.height=3)
obj_fgf1_neurons %>%
subset(cells = agrp_cells) %>%
VlnPlot(features= markers %>% filter(cluster == 'Agrp.pos') %>% head(10) %>% pull(gene), ncol=10)

In [ ]:
obj_fgf1_neurons %>% `[[` %>% head

In [ ]:
coord_comparison_df %>%
filter(label == 'Agrp') %>%
group_by(sample_name, treatment, polarity) %>%
summarise(n = n())

In [ ]:
coord_comparison_df %>%
filter(label == 'Agrp') %>%
group_by(sample_name, treatment, polarity) %>%
summarise(n = n()) %>%
filter(polarity == 'neg')

In [ ]:
coord_comparison_df %>%
filter(label == 'MOL') %>%
group_by(sample_name, treatment, polarity) %>%
summarise(n = n()) %>%
filter(polarity == 'pos')

In [ ]:
coord_comparison_df %>%
# filter(label == 'Rgs16_Dlx1') %>%
group_by(sample_name, treatment, polarity) %>%
summarise(n = n()) %>%
ungroup %>%
  pivot_wider(names_from = polarity, 
              values_from = n,
              names_prefix = "polarity_")

In [ ]:
obj_merged = qs::qread('_targets/objects/obj_merged')
obj_merged

In [ ]:
obj_merged %>% `[[` %>% group_by(sample_name) %>% summarise(n=n())

In [ ]:
xe_obj_cca_td_class = qs::qread('_targets/objects/xe_obj_cca_td_class')
xe_obj_cca_td_class

In [ ]:
xe_obj_cca_td_class %>% `[[` %>% group_by(sample_name) %>% summarise(n=n())